보스턴 데이터셋의 특징 출력

In [1]:
from sklearn.datasets import load_boston

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

dataset = load_boston() # 데이터셋을 불러옴
print(dataset.keys())   # 데이터셋의 키(요소들의 이름)를 출력

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

# 데이터의 구성요소 확인

In [ ]:
import pandas as pd

from sklearn.datasets import load_boston

dataset = load_boston()
dataFrame = pd.DataFrame(dataset["data"]) # ❶ 데이터셋의 데이터 불러오기
dataFrame.columns = dataset["feature_names"] # ❷ 특징의 이름 불러오기
dataFrame["target"] = dataset["target"] # ❸ 데이터 프레임에 정답을 추가

print(dataFrame.head()) # ➍ 데이터프레임을 요약해서 출력

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  target  
0     15.3  396.90   4.98    24.0  
1     17.8  396.90   9.14    21.6  
2     17.8  392.83   4.03    34.7  
3     18.7  394.63   2.94    33.4  
4     18.7  396.90   5.33    36.2  


# 선형회귀를 위한 MLP모델의 설계

In [ ]:
import torch
import torch.nn as nn

from torch.optim.adam import Adam


# ❶ 모델 정의
model = nn.Sequential(
   nn.Linear(13, 100),
   nn.ReLU(),
   nn.Linear(100, 1)
)

X = dataFrame.iloc[:, :13].values # ❷ 정답을 제외한 특징을 X에 입력
Y = dataFrame["target"].values    # 데이터프레임의 target의 값을 추출

batch_size = 100
learning_rate = 0.001

# ❸ 가중치를 수정하기 위한 최적화 정의
optim = Adam(model.parameters(), lr=learning_rate)


# 에포크 반복
for epoch in range(200):

   # 배치 반복
   for i in range(len(X)//batch_size):
       start = i*batch_size      # ➍ 배치 크기에 맞게 인덱스를 지정
       end = start + batch_size


       # 파이토치 실수형 텐서로 변환
       x = torch.FloatTensor(X[start:end])
       y = torch.FloatTensor(Y[start:end])

       optim.zero_grad() # ❺ 가중치의 기울기를 0으로 초기화
       preds = model(x)  # ❻ 모델의 예측값 계산
       loss = nn.MSELoss()(preds, y) # ❼ MSE 손실 계산
       loss.backward()
       optim.step()

   if epoch % 20 == 0:
       print(f"epoch{epoch} loss:{loss.item()}")

epoch0 loss:63.797157287597656
epoch20 loss:45.051475524902344
epoch40 loss:40.557090759277344
epoch60 loss:39.79398727416992
epoch80 loss:39.45023727416992
epoch100 loss:39.383544921875
epoch120 loss:39.04169845581055
epoch140 loss:38.85562515258789
epoch160 loss:38.72293472290039
epoch180 loss:38.49763488769531


# 모델 성능 평가

In [ ]:
prediction = model(torch.FloatTensor(X[0, :13]))
real = Y[0]
print(f"prediction:{prediction.item()} real:{real}")

prediction:25.561281204223633 real:24.0
